In [2]:
import cv2
import time
import threading
import multiprocessing as mp
import matplotlib.pyplot as plt

from situtils import FPSTimes

In [4]:
class WebcamStream(FPSTimes):
    # check camera output formats with FFMPEG
    # https://stackoverflow.com/questions/15301608/how-to-query-a-webcams-output-formats
    #
    # cd D:\software\ffmpeg
    # ffmpeg -list_devices true -f dshow -i dummy
    # ffmpeg -list_options true -f dshow -i video="HD USB Camera"
    #
    # Also
    # https://stackoverflow.com/questions/39308664/opencv-cant-set-mjpg-compression-for-usb-camera
    
    default_cfg = {
        'source': 0,
        'frame_width': 1024,
        'frame_height': 768,
        'fps': 25,
        'api': '',  # 700 -> cv2.CAP_DSHOW
        'verbose': True,
    }
    
    def __init__(self, cfg):
        super(WebcamStream, self).__init__()
        
        self.cfg = cfg
        self.frame = None
        self.frame_with_infos = None
        self.stopped = False
        
        # preparing a MPEG video stream
        self.stream = cv2.VideoCapture(cfg['source'], cfg['api']) if cfg['api'] else cv2.VideoCapture(cfg['source'])
        self.stream.set(cv2.CAP_PROP_FPS, cfg['fps'])
        time.sleep(1.5)  # this helps to keep the FPS stable
        #self.stream.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
        self.stream.set(cv2.CAP_PROP_FRAME_WIDTH, cfg['frame_width'])
        self.stream.set(cv2.CAP_PROP_FRAME_HEIGHT, cfg['frame_height'])
        self.stream.set(cv2.CAP_PROP_FPS, cfg['fps'])
        self.stream.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))  # cv2.VideoWriter_fourcc('H', '2', '6', '4')

    def start(self):
        self._th = threading.Thread(target=self.update, args=())
        self._th.start()

    def stop(self):
        self.stopped = True
        time.sleep(0.3)   # wait until device is released
        self._th.join()
        print('Camera released')
            
    def update(self):
        x_res = self.stream.get(cv2.CAP_PROP_FRAME_WIDTH)
        y_res = self.stream.get(cv2.CAP_PROP_FRAME_HEIGHT)
        fps = self.stream.get(cv2.CAP_PROP_FPS)
        print('Webcam stream %s:%s at %.2f FPS started' % (x_res, y_res, fps))
        
        while not self.stopped:
            (self.grabbed, self.frame) = self.stream.read()
            self.count()  # count FPS
            #cv2.waitKey(50)
            
        self.stream.release()
            
    def read(self):
        return self.frame

### Testing streaming

In [3]:
vs = WebcamStream(WebcamStream.default_cfg)
vs.start()  # stream runs in a separate thread

try:
    while True:
        frame = vs.read()
        if frame is not None:
            frame = cv2.putText(frame, '%.2f FPS' % vs.get_avg_fps(), (10, 20), 
                        cv2.FONT_HERSHEY_DUPLEX, .5, (255, 255, 255))
            cv2.imshow('Webcam', frame)

        k = cv2.waitKey(33)
        if k == ord('q'):
            break

finally:
    cv2.destroyAllWindows()
    vs.stop()

Webcam stream 960.0:720.0 at 20.00 FPS started
Camera released
